<a href="https://colab.research.google.com/github/Youxise/Tifinagh-MNIST-Classification/blob/main/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Path to your dataset (updated to reflect mounted Drive)
data_dir = '/content/drive/My Drive/Colab Notebooks/img'

# Parameters
img_height, img_width = 128, 128  # Adjust based on your image size
batch_size = 32

# Data Augmentation with additional techniques
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,       # Randomly rotate images by up to 30 degrees
    width_shift_range=0.2,   # Randomly shift images horizontally
    height_shift_range=0.2,  # Randomly shift images vertically
    shear_range=0.2,         # Shear transformation
    zoom_range=0.2,          # Zoom into images
    horizontal_flip=True,    # Flip images horizontally
    brightness_range=[0.8, 1.2],  # Random brightness
    validation_split=0.2     # Use 20% for validation
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Check the class labels (i.e., one-hot encoded labels)
class_indices = train_generator.class_indices
print("Class labels (one-hot encoded):", class_indices)

Mounted at /content/drive
Found 19257 images belonging to 33 classes.
Found 4807 images belonging to 33 classes.
Class labels (one-hot encoded): {'ⴰ': 0, 'ⴱ': 1, 'ⴳ': 2, 'ⴳⵯ': 3, 'ⴷ': 4, 'ⴹ': 5, 'ⴻ': 6, 'ⴼ': 7, 'ⴽ': 8, 'ⴽⵯ': 9, 'ⵀ': 10, 'ⵃ': 11, 'ⵄ': 12, 'ⵅ': 13, 'ⵇ': 14, 'ⵉ': 15, 'ⵊ': 16, 'ⵍ': 17, 'ⵎ': 18, 'ⵏ': 19, 'ⵓ': 20, 'ⵔ': 21, 'ⵕ': 22, 'ⵖ': 23, 'ⵙ': 24, 'ⵚ': 25, 'ⵛ': 26, 'ⵜ': 27, 'ⵟ': 28, 'ⵡ': 29, 'ⵢ': 30, 'ⵣ': 31, 'ⵥ': 32}


In [ ]:

# Define a smaller custom CNN model
model1 = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(class_indices), activation='softmax')
])

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load MobileNetV2 with weights and disable top layer
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')

# Unfreeze last 15 layers
for layer in base_model.layers[:-15]:
    layer.trainable = False

# Add new layers on top
model2 = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(len(class_indices), activation='softmax')
])

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


402/402 ━━━━━━━━━━━━━━━━━━━━ 3941s 10s/step - accuracy: 0.4200 - loss: 2.0227 - val_accuracy: 0.7187 - val_loss: 0.8951
Epoch 2/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 133s 221ms/step - accuracy: 0.7738 - loss: 0.7286 - val_accuracy: 0.7623 - val_loss: 0.7424
Epoch 3/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 89s 219ms/step - accuracy: 0.8219 - loss: 0.5579 - val_accuracy: 0.8367 - val_loss: 0.5138
Epoch 4/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 140s 215ms/step - accuracy: 0.8492 - loss: 0.4523 - val_accuracy: 0.8385 - val_loss: 0.4893
Epoch 5/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 142s 216ms/step - accuracy: 0.8660 - loss: 0.3976 - val_accuracy: 0.8622 - val_loss: 0.4660
Epoch 6/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 142s 215ms/step - accuracy: 0.8784 - loss: 0.3701 - val_accuracy: 0.8553 - val_loss: 0.4371
Epoch 7/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 88s 216ms/step - accuracy: 0.8862 - loss: 0.3461 - val_accuracy: 0.8749 - val_loss: 0.3941
Epoch 8/10
402/402 ━━━━━━━━━━━━━━━━━━━━ 143s 218ms/step - accuracy: 0.8911 - loss: 0.3240 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Adjust the learning rate and implement early stopping
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#model2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history1 = model1.fit(
    train_generator,
    epochs=10,  # Increase epochs for early stopping
    validation_data=validation_generator,
    callbacks=[early_stopping]
)

#history2 = model2.fit(
#    train_generator,
#    epochs=50,  # Increase epochs for early stopping
#    validation_data=validation_generator,
#    callbacks=[early_stopping]
#)


Epoch 1/10
190/602 ━━━━━━━━━━━━━━━━━━━━ 2:37:09 23s/step - accuracy: 0.0684 - loss: 3.4221

KeyboardInterrupt: 

In [ ]:
#model = tf.keras.models.load_model('my_model.h5')

# Evaluate the model on the validation set
validation_loss, validation_accuracy = model1.evaluate(validation_generator)
print(f"Validation Loss: {validation_loss}, Validation Accuracy: {validation_accuracy}")

# Predictions on validation data
y_pred = model1.predict(validation_generator)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = validation_generator.classes

# Get class labels
class_labels = list(class_indices.keys())

# Classification report
print("Classification Report:\n", classification_report(y_true, y_pred_classes, target_names=class_labels))

# Confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred_classes))

